##### Imports and Settings

In [27]:
import pandas as pd
import numpy as np
import requests
from functools import reduce
import matplotlib.pyplot as plt
import pickle
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
import sys
sys.path.append("../../../Functions and Dictionaries") # Adds higher directory to python modules path
import geodict
GNRC = geodict.GNRC
KY = geodict.KY
censusplaces = geodict.censusplaces
tofullcensus = geodict.tofullcensus
geotogeoid = geodict.geotogeoid
shorttnplaces = geodict.shorttnplaces
shortkyplaces = geodict.shortkyplaces
GNRC = geodict.GNRC
KY = geodict.KY
censusplaces = geodict.censusplaces
import tbrstuff
communitycollegescounties = tbrstuff.communitycollegescounties
tcatcampuscounties = tbrstuff.tcatcampuscounties
import sqlite3 as sq
#functions
def percent(x, y):
        try:
            return ((x/y)*100)
        except ZeroDivisionError:
            return 0
def percentchange(x, y):
    try:
        return ((x - y)*100/y)
    except ZeroDivisionError:
        return 0
def realchange(x, y):
    return x-y
#calculate real and percent change between all columns for all possible time frames
def calculate_changes(df, columns, time_frames, years):
    for column in columns:
        for time_frame in time_frames:
            start_year, end_year = time_frame.split('-')
            df[f'{column} % Change', 'None', f'{time_frame}'] = percentchange(df[(column, int(end_year), 'None')], df[(column, int(start_year), 'None')])
            df[f'{column} Change', 'None', f'{time_frame}'] = (df[(column, int(end_year), 'None')] - df[(column, int(start_year), 'None')])

    return df
#generate all possible time frames from a list of years
def generate_time_frames(years):
    time_frames = []
    for i in range(len(years)-1):
        for j in range(i+1, len(years)):
            time_frames.append(f"{years[i]}-{years[j]}")
    return time_frames

In [45]:
cip = pd.read_csv('../../../Heirarchies Folder/data/CIPCrosswalk.csv', dtype = str)

In [46]:
cip.head()

,Program Family,Two Digit,Program Category,Four Digit,Program,Six Digit
0,Agricultural/Animal/Plant/Veterinary Science a...,01,"Agriculture, General",01.00,"Agriculture, General",01.0000
1,Agricultural/Animal/Plant/Veterinary Science a...,01,Agricultural Business and Management,01.01,"Agricultural Business and Management, General",01.0101
2,Agricultural/Animal/Plant/Veterinary Science a...,01,Agricultural Business and Management,01.01,Agribusiness/Agricultural Business Operations,01.0102
3,Agricultural/Animal/Plant/Veterinary Science a...,01,Agricultural Business and Management,01.01,Agricultural Economics,01.0103
4,Agricultural/Animal/Plant/Veterinary Science a...,01,Agricultural Business and Management,01.01,Farm/Farm and Ranch Management,01.0104


In [36]:
cip.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2173 entries, 0 to 2172
Data columns (total 6 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Program Family    2173 non-null   object
 1   Two Digit         2173 non-null   object
 2   Program Category  2173 non-null   object
 3   Four Digit        2173 non-null   object
 4   Program           2173 non-null   object
 5   Six Digit         2173 non-null   object
dtypes: object(6)
memory usage: 102.0+ KB


In [9]:
recent = pd.read_csv('../../Data Downloads/TBR_TCAT_20222023.csv', dtype = str)

In [10]:
recent.head()

,award_count,TCAT_School,Award_Type,ReportYr,location_name,cip
0,12,Athens,Certificate,2022-23,Main Campus,480508
1,1,Athens,Certificate,2022-23,Main Campus,480501
2,1,Athens,Certificate,2022-23,Main Campus,470604
3,4,Athens,Diploma,2022-23,Main Campus,520402
4,5,Athens,Diploma,2022-23,Main Campus,470603


In [11]:
comm = pd.read_csv('../../Data Downloads/TBR_CommunityColleges_20192020_20222023.csv', dtype = str)

In [12]:
comm.head()

,award_count,COLLEGE,Award_Type,Report_Year,CIP
0,2,Chattanooga,Certificate,2019 - 2020,110801
1,7,Chattanooga,Certificate,2019 - 2020,150000
2,15,Chattanooga,Certificate,2019 - 2020,150406
3,11,Chattanooga,Certificate,2019 - 2020,150612
4,4,Chattanooga,Certificate,2019 - 2020,150614


In [128]:
old = pd.read_csv('../../Data Downloads/TBR_TCAT_old.csv', dtype = str)

In [129]:
old.head()

,Total Awards,Award Type,Year,Institution,Campus,County,FIPS,Two Digit,Six Digit,Four Digit
0,2,Certificate,2019-20,Knoxville,Anderson County Career and Technical Center,"Anderson County, Tennessee",47001,48,48.0508,48.05
1,7,Diploma,2019-20,Knoxville,Anderson County Career and Technical Center,"Anderson County, Tennessee",47001,48,48.0508,48.05
2,3,Diploma,2019-20,Knoxville,Anderson County Career and Technical Center,"Anderson County, Tennessee",47001,48,48.0501,48.05
3,6,Diploma,2019-20,Knoxville,Oak Ridge High School,"Anderson County, Tennessee",47001,48,48.0508,48.05
4,10,Certificate,2020-21,Knoxville,Anderson County Career and Technical Center,"Anderson County, Tennessee",47001,48,48.0508,48.05


In [130]:
old.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2308 entries, 0 to 2307
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Total Awards  2308 non-null   object
 1   Award Type    2308 non-null   object
 2   Year          2308 non-null   object
 3   Institution   2308 non-null   object
 4   Campus        2308 non-null   object
 5   County        2308 non-null   object
 6   FIPS          2308 non-null   object
 7   Two Digit     2308 non-null   object
 8   Six Digit     2308 non-null   object
 9   Four Digit    2308 non-null   object
dtypes: object(10)
memory usage: 180.4+ KB


In [132]:
#cip = cip.drop(columns = ['Four Digit', 'Two Digit'])

In [133]:
data = old.merge(cip, on = 'Six Digit')

In [134]:
data.head()

,Total Awards,Award Type,Year,Institution,Campus,County,FIPS,Two Digit,Six Digit,Four Digit,Program Family,Program Category,Program
0,2,Certificate,2019-20,Knoxville,Anderson County Career and Technical Center,"Anderson County, Tennessee",47001,48,48.0508,48.05,Precision Production,Precision Metal Working,Welding Technology/Welder
1,7,Diploma,2019-20,Knoxville,Anderson County Career and Technical Center,"Anderson County, Tennessee",47001,48,48.0508,48.05,Precision Production,Precision Metal Working,Welding Technology/Welder
2,6,Diploma,2019-20,Knoxville,Oak Ridge High School,"Anderson County, Tennessee",47001,48,48.0508,48.05,Precision Production,Precision Metal Working,Welding Technology/Welder
3,10,Certificate,2020-21,Knoxville,Anderson County Career and Technical Center,"Anderson County, Tennessee",47001,48,48.0508,48.05,Precision Production,Precision Metal Working,Welding Technology/Welder
4,8,Diploma,2020-21,Knoxville,Anderson County Career and Technical Center,"Anderson County, Tennessee",47001,48,48.0508,48.05,Precision Production,Precision Metal Working,Welding Technology/Welder


In [135]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2304 entries, 0 to 2303
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Total Awards      2304 non-null   object
 1   Award Type        2304 non-null   object
 2   Year              2304 non-null   object
 3   Institution       2304 non-null   object
 4   Campus            2304 non-null   object
 5   County            2304 non-null   object
 6   FIPS              2304 non-null   object
 7   Two Digit         2304 non-null   object
 8   Six Digit         2304 non-null   object
 9   Four Digit        2304 non-null   object
 10  Program Family    2304 non-null   object
 11  Program Category  2304 non-null   object
 12  Program           2304 non-null   object
dtypes: object(13)
memory usage: 252.0+ KB


In [136]:
test = data

In [137]:
cols = list(test.columns)
cols.remove('Total Awards')

In [138]:
test.head()

,Total Awards,Award Type,Year,Institution,Campus,County,FIPS,Two Digit,Six Digit,Four Digit,Program Family,Program Category,Program
0,2,Certificate,2019-20,Knoxville,Anderson County Career and Technical Center,"Anderson County, Tennessee",47001,48,48.0508,48.05,Precision Production,Precision Metal Working,Welding Technology/Welder
1,7,Diploma,2019-20,Knoxville,Anderson County Career and Technical Center,"Anderson County, Tennessee",47001,48,48.0508,48.05,Precision Production,Precision Metal Working,Welding Technology/Welder
2,6,Diploma,2019-20,Knoxville,Oak Ridge High School,"Anderson County, Tennessee",47001,48,48.0508,48.05,Precision Production,Precision Metal Working,Welding Technology/Welder
3,10,Certificate,2020-21,Knoxville,Anderson County Career and Technical Center,"Anderson County, Tennessee",47001,48,48.0508,48.05,Precision Production,Precision Metal Working,Welding Technology/Welder
4,8,Diploma,2020-21,Knoxville,Anderson County Career and Technical Center,"Anderson County, Tennessee",47001,48,48.0508,48.05,Precision Production,Precision Metal Working,Welding Technology/Welder


In [139]:
result_df = test.pivot(index = ['Year', 'Institution', 'Campus', 'County', 'FIPS', 'Two Digit', 'Six Digit', 
                                'Four Digit', 'Program Family', 'Program Category', 'Program'], 
                                columns='Award Type', values= 'Total Awards')

In [140]:
result_df.head()

Award Type                                                                                                                                                                                                                                             Certificate  \
Year    Institution Campus      County                   FIPS  Two Digit Six Digit Four Digit Program Family                                     Program Category                                   Program                                                          
2019-20 Athens      Main Campus McMinn County, Tennessee 47107 11        11.0901   11.09      Computer and Information Sciences and Support S... Computer Systems Networking and Telecommunications Computer Systems Networking and Telecommunications         NaN   
                                                               47        47.0303   47.03      Mechanic and Repair Technologies/Technicians       Heavy/Industrial Equipment Maintenance Technolo... Industrial Mechanics and Maintenance Technology...           8   
                                                                         47.0603   47.06      Mechanic and Repair Technologies/Technicians       Vehicle Maintenance and Repair Technologies/Tec... Autobody/Collision and Repair Technology/Techni...           1   
                                                                         47.0604   47.06      Mechanic and Repair Technologies/Technicians       Vehicle Maintenance and Repair Technologies/Tec... Automobile/Automotive Mechanics Technology/Tech...           7   
                                                               48        48.0501   48.05      Precision Production                               Precision Metal Working                            Machine Tool Technology/Machinist                          NaN   

Award Type                                                                                                                                                                                                                                             Diploma  
Year    Institution Campus      County                   FIPS  Two Digit Six Digit Four Digit Program Family                                     Program Category                                   Program                                                     
2019-20 Athens      Main Campus McMinn County, Tennessee 47107 11        11.0901   11.09      Computer and Information Sciences and Support S... Computer Systems Networking and Telecommunications Computer Systems Networking and Telecommunications      10  
                                                               47        47.0303   47.03      Mechanic and Repair Technologies/Technicians       Heavy/Industrial Equipment Maintenance Technolo... Industrial Mechanics and Maintenance Technology...       8  
                                                                         47.0603   47.06      Mechanic and Repair Technologies/Technicians       Vehicle Maintenance and Repair Technologies/Tec... Autobody/Collision and Repair Technology/Techni...      10  
                                                                         47.0604   47.06      Mechanic and Repair Technologies/Technicians       Vehicle Maintenance and Repair Technologies/Tec... Automobile/Automotive Mechanics Technology/Tech...       4  
                                                               48        48.0501   48.05      Precision Production                               Precision Metal Working                            Machine Tool Technology/Machinist                       11

In [141]:
result_df.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 1431 entries, ('2019-20', 'Athens', 'Main Campus', 'McMinn County, Tennessee', '47107', '11', '11.0901', '11.09', 'Computer and Information Sciences and Support Services', 'Computer Systems Networking and Telecommunications', 'Computer Systems Networking and Telecommunications') to ('2021-22', 'Shelbyville', 'Middle Tennessee Education Center', 'Bedford County, Tennessee', '47003', '11', '11.1002', '11.10', 'Computer and Information Sciences and Support Services', 'Computer/Information Technology Administration and Management', 'System, Networking, and LAN/WAN Management/Manager')
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Certificate  1024 non-null   object
 1   Diploma      1280 non-null   object
dtypes: object(2)
memory usage: 56.9+ KB


In [142]:
df = result_df.reset_index(drop = False)

In [143]:
df.head()

Award Type,Year,Institution,Campus,County,FIPS,Two Digit,Six Digit,Four Digit,Program Family,Program Category,Program,Certificate,Diploma
0,2019-20,Athens,Main Campus,"McMinn County, Tennessee",47107,11,11.0901,11.09,Computer and Information Sciences and Support ...,Computer Systems Networking and Telecommunicat...,Computer Systems Networking and Telecommunicat...,NaN,10
1,2019-20,Athens,Main Campus,"McMinn County, Tennessee",47107,47,47.0303,47.03,Mechanic and Repair Technologies/Technicians,Heavy/Industrial Equipment Maintenance Technol...,Industrial Mechanics and Maintenance Technolog...,8,8
2,2019-20,Athens,Main Campus,"McMinn County, Tennessee",47107,47,47.0603,47.06,Mechanic and Repair Technologies/Technicians,Vehicle Maintenance and Repair Technologies/Te...,Autobody/Collision and Repair Technology/Techn...,1,10
3,2019-20,Athens,Main Campus,"McMinn County, Tennessee",47107,47,47.0604,47.06,Mechanic and Repair Technologies/Technicians,Vehicle Maintenance and Repair Technologies/Te...,Automobile/Automotive Mechanics Technology/Tec...,7,4
4,2019-20,Athens,Main Campus,"McMinn County, Tennessee",47107,48,48.0501,48.05,Precision Production,Precision Metal Working,Machine Tool Technology/Machinist,NaN,11


In [144]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1431 entries, 0 to 1430
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Year              1431 non-null   object
 1   Institution       1431 non-null   object
 2   Campus            1431 non-null   object
 3   County            1431 non-null   object
 4   FIPS              1431 non-null   object
 5   Two Digit         1431 non-null   object
 6   Six Digit         1431 non-null   object
 7   Four Digit        1431 non-null   object
 8   Program Family    1431 non-null   object
 9   Program Category  1431 non-null   object
 10  Program           1431 non-null   object
 11  Certificate       1024 non-null   object
 12  Diploma           1280 non-null   object
dtypes: object(13)
memory usage: 145.5+ KB


In [145]:
df = df.fillna(0)

In [146]:
cols = ['Certificate', 'Diploma']
df[cols] = df[cols].astype(int)

In [147]:
df['Total Awards'] = df['Certificate'] + df['Diploma']

In [148]:
df.head()

Award Type,Year,Institution,Campus,County,FIPS,Two Digit,Six Digit,Four Digit,Program Family,Program Category,Program,Certificate,Diploma,Total Awards
0,2019-20,Athens,Main Campus,"McMinn County, Tennessee",47107,11,11.0901,11.09,Computer and Information Sciences and Support ...,Computer Systems Networking and Telecommunicat...,Computer Systems Networking and Telecommunicat...,0,10,10
1,2019-20,Athens,Main Campus,"McMinn County, Tennessee",47107,47,47.0303,47.03,Mechanic and Repair Technologies/Technicians,Heavy/Industrial Equipment Maintenance Technol...,Industrial Mechanics and Maintenance Technolog...,8,8,16
2,2019-20,Athens,Main Campus,"McMinn County, Tennessee",47107,47,47.0603,47.06,Mechanic and Repair Technologies/Technicians,Vehicle Maintenance and Repair Technologies/Te...,Autobody/Collision and Repair Technology/Techn...,1,10,11
3,2019-20,Athens,Main Campus,"McMinn County, Tennessee",47107,47,47.0604,47.06,Mechanic and Repair Technologies/Technicians,Vehicle Maintenance and Repair Technologies/Te...,Automobile/Automotive Mechanics Technology/Tec...,7,4,11
4,2019-20,Athens,Main Campus,"McMinn County, Tennessee",47107,48,48.0501,48.05,Precision Production,Precision Metal Working,Machine Tool Technology/Machinist,0,11,11


In [149]:
df = df.rename_axis(None, axis=1)

In [150]:
df.head()

,Year,Institution,Campus,County,FIPS,Two Digit,Six Digit,Four Digit,Program Family,Program Category,Program,Certificate,Diploma,Total Awards
0,2019-20,Athens,Main Campus,"McMinn County, Tennessee",47107,11,11.0901,11.09,Computer and Information Sciences and Support ...,Computer Systems Networking and Telecommunicat...,Computer Systems Networking and Telecommunicat...,0,10,10
1,2019-20,Athens,Main Campus,"McMinn County, Tennessee",47107,47,47.0303,47.03,Mechanic and Repair Technologies/Technicians,Heavy/Industrial Equipment Maintenance Technol...,Industrial Mechanics and Maintenance Technolog...,8,8,16
2,2019-20,Athens,Main Campus,"McMinn County, Tennessee",47107,47,47.0603,47.06,Mechanic and Repair Technologies/Technicians,Vehicle Maintenance and Repair Technologies/Te...,Autobody/Collision and Repair Technology/Techn...,1,10,11
3,2019-20,Athens,Main Campus,"McMinn County, Tennessee",47107,47,47.0604,47.06,Mechanic and Repair Technologies/Technicians,Vehicle Maintenance and Repair Technologies/Te...,Automobile/Automotive Mechanics Technology/Tec...,7,4,11
4,2019-20,Athens,Main Campus,"McMinn County, Tennessee",47107,48,48.0501,48.05,Precision Production,Precision Metal Working,Machine Tool Technology/Machinist,0,11,11


In [151]:
df['Institution Type'] = 'TCAT'

In [152]:
df = df[['Year', 'Program', 'Six Digit', 'Program Category', 'Four Digit', 'Program Family', 'Two Digit', 'Institution Type', 'Institution', 
         'Campus', 'County', 'FIPS', 'Certificate', 'Diploma', 'Total Awards'
        ]]

In [153]:
df.head()

,Year,Program,Six Digit,Program Category,Four Digit,Program Family,Two Digit,Institution Type,Institution,Campus,County,FIPS,Certificate,Diploma,Total Awards
0,2019-20,Computer Systems Networking and Telecommunicat...,11.0901,Computer Systems Networking and Telecommunicat...,11.09,Computer and Information Sciences and Support ...,11,TCAT,Athens,Main Campus,"McMinn County, Tennessee",47107,0,10,10
1,2019-20,Industrial Mechanics and Maintenance Technolog...,47.0303,Heavy/Industrial Equipment Maintenance Technol...,47.03,Mechanic and Repair Technologies/Technicians,47,TCAT,Athens,Main Campus,"McMinn County, Tennessee",47107,8,8,16
2,2019-20,Autobody/Collision and Repair Technology/Techn...,47.0603,Vehicle Maintenance and Repair Technologies/Te...,47.06,Mechanic and Repair Technologies/Technicians,47,TCAT,Athens,Main Campus,"McMinn County, Tennessee",47107,1,10,11
3,2019-20,Automobile/Automotive Mechanics Technology/Tec...,47.0604,Vehicle Maintenance and Repair Technologies/Te...,47.06,Mechanic and Repair Technologies/Technicians,47,TCAT,Athens,Main Campus,"McMinn County, Tennessee",47107,7,4,11
4,2019-20,Machine Tool Technology/Machinist,48.0501,Precision Metal Working,48.05,Precision Production,48,TCAT,Athens,Main Campus,"McMinn County, Tennessee",47107,0,11,11


In [154]:
df.to_csv('currentTBRforstorage.csv', index = False)

In [155]:
df = pd.read_csv('../../Data Downloads/TBR_TCATS_201920_202122.csv', dtype = str)

In [156]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1431 entries, 0 to 1430
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Year              1431 non-null   object
 1   Program           1431 non-null   object
 2   Six Digit         1431 non-null   object
 3   Program Category  1431 non-null   object
 4   Four Digit        1431 non-null   object
 5   Program Family    1431 non-null   object
 6   Two Digit         1431 non-null   object
 7   Institution Type  1431 non-null   object
 8   Institution       1431 non-null   object
 9   Campus            1431 non-null   object
 10  County            1431 non-null   object
 11  FIPS              1431 non-null   object
 12  Certificate       1431 non-null   object
 13  Diploma           1431 non-null   object
 14  Total Awards      1431 non-null   object
dtypes: object(15)
memory usage: 167.8+ KB
